# Отчет по заданию

## Задание: 
<br/>

***Создать дашборд по звонкам в CallCenter***

<br/>
Пример визуализации Dashboard:
<br/>

![001.png](pics/001.png)
![002.png](pics/002.png)

<br/>

***Витрина данных:*** файл SLA_CONTACT_CENTRE.csv

***Описание витрины:***

№ | Наименование | Описание
:-|:-------------|:---------
1 | REPORT       | Тип телефонии с агрегацией. Например, Avaya By 15min – тип Avaya с разбивкой 15 минут
2 | DT           | Отчетная дата
3 | LD      | Дата загрузки в Хранилище данных
4 | DTTM | Дата и время периода агрегации
5 | SKILL_NM | Наименование скилла
6 | CALLS | Количество звонков
7 | ACCEPTED_CALLS | Количество принятых звонков
8 | RESET_CALLS | Количество непринятых звонков
9 | SLA | Нагрузка на операторов
10 | MAX_CALL_DT | Максимальное время звонка

***Параметры дашборда:***

1)	Отчетная дата (DT) – дата, на которую формируется отчет

***Блок Актуальность данных:***

1)	Телефония (Avaya и/или GENESYS)
2)	Максимальное время звонка (MAX_CALL_DT)
3)	Время загрузки данных по звонкам в Хранилище данных (LD)
4)	Отставание в минутах: разница от текущего времени и максимального времени звонка (пункт 2)

***Блок Отчетная дата*** – текущая дата формирования отчета из фильтра

***Блок Объем входящих звонков*** – общее количество звонков

***Блок SLA первый ответ в %*** (SLA)

***Блок количество непринятых звонков*** (RESET_CALLS)

***Блок SLA КЦ*** – график  с отображением объема звонков и нагрузки на операторов  за последние два дня (от отчетной даты).

***Блок Calls By Skills by Hour*** – объем звонков в разрезе скиллов по часам за текущие сутки (отчетная дата).



## Решение Задания:
<br/>
Создаю базу, таблицу, и наполню их данными:

``` mysql
CREATE DATABASE IF NOT EXISTS ccenter;

CREATE TABLE ccenter.sla_contact_center
(
REPORT String,
DT Date,
LD DateTime64,
DTTM DateTime64,
SKILL_NM String,
CALLS Int64,
ACCEPTED_CALLS Int64,
RESET_CALLS Int64,
SLA Float64,
MAX_CALL_DT Nullable(DateTime64)
) ENGINE = MergeTree ORDER BY (SKILL_NM) SETTINGS index_granularity = 8192;
```
<br/>
<br/>
Данные из CSV файла оказались немного разноформатными. Поэтому их надо было причесать. Для этого я использовал следующий код:

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = None # display(df)
pd.options.display.max_rows = None
pd.options.display.min_rows = None

import pandas as pd
from clickhouse_driver import Client

df = pd.read_csv('./SLA_CONTACT_CENTRE.csv', sep=';')

df['DT'] = pd.to_datetime(df['DT'])
df['LD'] = pd.to_datetime(df['LD'])
df['DTTM'] = pd.to_datetime(df['DTTM'])
df['MAX_CALL_DT'] = df['MAX_CALL_DT'].fillna(0)
df['MAX_CALL_DT'] = pd.to_datetime(df['MAX_CALL_DT'])
df['REPORT'] = df['REPORT'].astype(str)
df['SKILL_NM'] = df['SKILL_NM'].astype(str)
df['SLA'] = pd.to_numeric(df['SLA'].str.replace("[,]", ".", regex=True),errors='coerce')
display(df.dtypes)
df.head(3)

/var/folders/xf/23jw26zs7r37__lj3ydbh14h0000gn/T/ipykernel_3259/2964235819.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


REPORT                    object
DT                datetime64[ns]
LD                datetime64[ns]
DTTM              datetime64[ns]
SKILL_NM                  object
CALLS                      int64
ACCEPTED_CALLS             int64
RESET_CALLS                int64
SLA                      float64
MAX_CALL_DT       datetime64[ns]
dtype: object

,REPORT,DT,LD,DTTM,SKILL_NM,CALLS,ACCEPTED_CALLS,RESET_CALLS,SLA,MAX_CALL_DT
0,Avaya By Day,2021-05-06,2021-09-09 17:43:00,2021-05-06 00:00:00,Itogo,52218,52008,210,0.962801,2021-05-06 23:58:51
1,Genesys By Day,2021-05-06,2021-08-05 23:59:00,2021-05-06 00:00:00,Itogo,367,367,0,1.000000,2021-05-06 08:18:21
2,Total By 15min,2021-05-06,2021-09-09 18:17:00,2021-05-06 08:30:00,Itogo,727,727,0,1.000000,2021-05-06 08:44:59


In [2]:
client = Client('10.8.10.119',
                database='ccenter',
                user='default'
                )
client.execute("INSERT INTO sla_contact_center VALUES", df.to_dict('records'))

2621

In [4]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/S_f2qV2_U00?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')


/Users/dmitrijkarpuhin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
<br/>
<br/>

<br/>
<br/>
Результирующая таблица на скрине внизу:
<br/>

![01.png](pics/01.png)